In [1]:
import tensorflow as tf # keras = 2.15.0
import pandas as pd # 2.1.3
import numpy as np # 1.26.2
import matplotlib.pyplot as plt # 3.8.2
from sklearn.preprocessing import MinMaxScaler,StandardScaler # 1.3.2
import ast # python = 3.10.0

In [3]:
def datetime2index(data,time_clo):
    # Converts dates to a 'datetime' class and sets it as an index
    data[time_clo] = pd.to_datetime(data[time_clo])
    data.set_index(time_clo, inplace=True)
    try:
        data = data.drop("Unnamed: 0",axis=1) # drop unnamed column
    except:
        return data
    return data

# load datas
data = pd.read_csv('./datas.csv')
# Load labels
monthly_labels = pd.read_csv("./monthly_merged_label.csv")
quarterly_labels = pd.read_csv("./Quarterly_merged_label.csv")

# Converts dates to a 'datetime' class and sets it as an index
data = datetime2index(data,'times')
# two kinds of labels
monthly_labels = datetime2index(monthly_labels,'observation_date')
quarterly_labels = datetime2index(quarterly_labels,'observation_date')

In [4]:
# Completion of missing dates
# 1. Create a DataFrame with all the dates.
all_dates = pd.date_range(start=data.index.min(), end=data.index.max(), freq='D')
all_dates_df = pd.DataFrame(index=all_dates)

# 2. Merging data
data = all_dates_df.join(data, how='left')
missing_values_per_column = data.isna().sum()
print('len of data:',len(data),'\nmissing values:\n',missing_values_per_column)
# Completion of statistics for missing dates with 0
data = data.fillna(0)
# the input feature columns
print(data.columns[4:])
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Prepare the inputs
input_data = []
for month, group in data.groupby(pd.Grouper(freq='M')): # M：Month；Q：Quarter
    # monthly_feature = group[['mean', 'median', 'max', 'min', 'std', 'rolling_mean', 'rolling_std', 'change_rate']].values
    monthly_feature = group[data.columns[4:]].values
    input_data.append(monthly_feature)

# Use pad_sequences for padding to ensure that the data is the same length for each month/quarter
input_data_padded = pad_sequences(input_data, padding='post', dtype='float32')
# Checking the shape of the filled data
print(input_data_padded.shape)

len of data: 2569 
missing values:
 sentiment_scores_1          651
sentiment_scores_2          651
sentiment_scores_3          651
sentiment_scores_4          651
sentiment_scores_1_mean    1226
sentiment_scores_1_std     1226
sentiment_scores_2_mean     674
sentiment_scores_2_std      674
sentiment_scores_3_mean     674
sentiment_scores_3_std      674
sentiment_scores_4_mean    1387
sentiment_scores_4_std     1387
dtype: int64
Index(['sentiment_scores_1_mean', 'sentiment_scores_1_std',
       'sentiment_scores_2_mean', 'sentiment_scores_2_std',
       'sentiment_scores_3_mean', 'sentiment_scores_3_std',
       'sentiment_scores_4_mean', 'sentiment_scores_4_std'],
      dtype='object')
(85, 31, 8)


In [5]:
from pandas.tseries.offsets import MonthEnd
from pandas.tseries.offsets import QuarterEnd
#  确定时间范围
start_date = data.index.min() # 开始时间
month_end_date = data.index.max()  + MonthEnd(1) # 用过去预测未来
quarter_end_date = data.index.max()  + QuarterEnd(1)
print(start_date,month_end_date,quarter_end_date)
# 筛选对应时间范围内的标签
monthly_labels = monthly_labels.loc[start_date:month_end_date]
quarterly_labels = quarterly_labels.loc[start_date:quarter_end_date]

2006-10-20 00:00:00 2013-11-30 00:00:00 2013-12-31 00:00:00


In [6]:
# Initialising a MinMaxScater
scaler = MinMaxScaler()
# select the predicted label
'''
在这里选择要预测的标签
'''
label = monthly_labels['CMRMT'] # 85
# label = quarterly_labels['GDP Growth Rate'] # 28 # GDP index
label_scaled = scaler.fit_transform(label.values.reshape(-1, 1))
# label_scaled = label.values.reshape(-1,1)
print(label_scaled.shape)

(85, 1)


In [7]:
# The datas and labels for model training
X = input_data_padded # for monthly labels
# X = input_data_padded[:28] # for quarterly labels : 缺少一个最新季度的标签所以不得不去掉最后一个月的特征
y = label_scaled

# 留出验证，划分训练集和测试集
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(68, 31, 8) (17, 31, 8)
(68, 1) (17, 1)


In [ ]:
# 使用模型进行预测:
import tensorflow as tf
# 指定保存模型的路径
model_path = './models/***.h5'
# 加载模型
loaded_model = tf.keras.models.load_model(model_path)
# 查看模型的结构
loaded_model.summary()
# 使用加载的模型进行预测
# loaded_model.predict(X/X_train/X_test)